In [2]:
pip install sb3-contrib
# 注意：如果安装失败，尝试关闭vpn

Could not fetch URL https://pypi.org/simple/sb3-contrib/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/sb3-contrib/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1125)'))) - skipping
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1125)'))) - skipping


ERROR: Could not find a version that satisfies the requirement sb3-contrib
ERROR: No matching distribution found for sb3-contrib


In [2]:
from sb3_contrib import MaskablePPO

In [3]:
import numpy as np

In [6]:
np.array([1,2,3])

array([1, 2, 3])

In [8]:
from stable_baselines3.common.env_checker import check_env
from gym_reversi import ReversiEnv

env = ReversiEnv(player_color='black', opponent = "random", board_size=8)
# It will check your custom environment and output additional warnings if needed
check_env(env)


D:\apps\Anaconda3\lib\site-packages\stable_baselines3\common\env_checker.py:38: UserWarning: It seems that your observation space  is an image but the upper and lower bounds are not in [0, 255]. Because the CNN policy normalize automatically the observation you may encounter issue if the values are not in that range.
  warnings.warn(
D:\apps\Anaconda3\lib\site-packages\stable_baselines3\common\env_checker.py:51: UserWarning: The minimal resolution for an image is 36x36 for the default `CnnPolicy`. You might need to use a custom features extractor cf. https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html
  warnings.warn(


In [5]:
??make_vec_env

In [6]:
??ActionMasker

### Maskable PPO

In [2]:
# import sys
import os
# current_path = os.getcwd()
# sys.path.append(current_path)
import os
import time
import gymnasium as gym
import numpy as np

from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack
# from stable_baselines3 import DQN, DDPG, A2C, PPO,SAC,TD3
from stable_baselines3 import PPO
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

import time
from gym_reversi import ReversiEnv
from custom_feature_extractor import CustomCNN

t0=time.time()

# There already exists an environment generator
# that will make and wrap atari environments correctly.
# Here we are also multi-worker training (n_envs=4 => 4 environments)
# vec_env = make_atari_env("PongNoFrameskip-v4", n_envs=4, seed=0)
# env = make_atari_env("BreakoutNoFrameskip-v4", seed=0)
# vec_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=4, 
# #                          seed=0
#                         )
# # Frame-stacking with 4 frames
# vec_env = VecFrameStack(vec_env, n_stack=4)

def mask_fn(env: gym.Env) -> np.ndarray:
    # Do whatever you'd like in this function to return the action mask
    # for the current env. In this example, we assume the env has a
    # helpful method we can rely on.
    return env.valid_action_mask()

# backbone= 'resnet'
backbone= 'cnn'
n_channels = 3


board_size=8
total_timesteps=10_0000
# PolicyModel = PPO
PolicyModel = MaskablePPO
# PolicyModel = TD3
n_envs = 4

greedy_rate=0
verbose = 0

tensorboard_log = f"models/ppo_{board_size}x{board_size}_{backbone}_debug/"
if not os.path.isdir(tensorboard_log):
    os.makedirs(tensorboard_log)
model_path = os.path.join(tensorboard_log, "model")
opponent_model_path="random"
# opponent_model_path=os.path.join(tensorboard_log, "opponent_model")


if opponent_model_path != "random":
    opponent_model = PolicyModel.load(opponent_model_path)
else:
    opponent_model = "random"

env = ReversiEnv(opponent=opponent_model, is_train=True, board_size=board_size, n_channels=4,
                 greedy_rate=greedy_rate, verbose=verbose)

# env = ActionMasker(env, mask_fn)  # Wrap to enable masking

vec_env = env
if n_envs > 1:
    # multi-worker training (n_envs=4 => 4 environments)
    vec_env = make_vec_env(ReversiEnv, n_envs=n_envs, seed=None,
                           env_kwargs={
                               "opponent": opponent_model,
                               "is_train": True,
                               "board_size": board_size,
                               "greedy_rate": greedy_rate,
                               "verbose": verbose},
                        )

    
policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=256,
                                   backbone=backbone,
                                   net_arch=[64, 64, 128],
#                                    net_arch=[32, 64, 64],
                                   kernel_size=3, 
                                   stride=1, 
                                   padding='same', 
                                   is_batch_norm=False),
    net_arch=[256, 256], 
    normalize_images=False
)
    
try:
    model = PolicyModel.load(model_path, env=vec_env)
except Exception:
    print(f"load model from self.model_path: {model_path} error")
    model = PolicyModel(MaskableActorCriticPolicy, vec_env,
                  policy_kwargs=policy_kwargs,
                  learning_rate=2.5e-4,  # learning_rate=2.5e-4,
                  ent_coef=0.01,
                  n_steps=64, # n_steps=128,
                  n_epochs=4,
                  batch_size=32, # batch_size=256,
                  gamma=0.9,
                  gae_lambda=0.9,
                  clip_range=0.2,
                  vf_coef=0.5,
                  verbose=1,
                  tensorboard_log=tensorboard_log)

t0 = time.time()
# model.learn(int(2e4))
model.learn(total_timesteps=total_timesteps)
model.save(model_path)
print(f"train time: {time.time()-t0}")

# vec_env = make_atari_env("BreakoutNoFrameskip-v4", n_envs=1)
# vec_env = VecFrameStack(vec_env, n_stack=4)


inplanes:4
planes:64
inplanes:64
planes:64
inplanes:64
planes:128
Logging to models/ppo_8x8_resnet_debug/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 30.1     |
|    ep_rew_mean     | 0.5      |
| time/              |          |
|    fps             | 206      |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 256      |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 30.1        |
|    ep_rew_mean          | 0.375       |
| time/                   |             |
|    fps                  | 141         |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 512         |
| train/                  |             |
|    approx_kl            | 0.062315006 |
|    clip_fraction        | 0.335       |
|    clip_range           | 0.2         |
|  

KeyboardInterrupt: 

In [127]:
0.85**50

0.00029576466371269896

In [3]:
tensorboard --logdir ./reversi/models/Reversi_ppo/PPO_7/ --port=6016

8

In [1]:
import torch
# from stable_baselines3 import PPO
from sb3_contrib.ppo_mask import MaskablePPO
PolicyModel = MaskablePPO

# model_path = "models/ppo_8x8_cnn_debug2/model_2600w"
model_path = "models/cnn_selfplay/model_2915w"
# model_path = "models/model_460w"
state_dict_path = model_path + '_state_dict.pt'

sb3_model = PolicyModel.load(model_path)

torch.save(sb3_model.policy.state_dict(), state_dict_path)

In [2]:
import torch
# from model_deploy.policies import CustomCNN, ActorCriticPolicy
from model_deploy.custom_feature_extractor import CustomCNN
from model_deploy.policies import MaskableActorCriticPolicy


backbone='cnn'
# backbone='res_net'

n_channels=3


features_extractor_kwargs=dict(features_dim=256,
                               backbone=backbone,
                               net_arch=[128, 128, 256],
#                                net_arch=[64, 128, 128],
                               kernel_size=3,
                               stride=1,
                               padding=1,
                               is_batch_norm=True
                               )

observation_space=(n_channels, 8, 8)
action_space = [observation_space[1]*observation_space[2]]

policy_model = MaskableActorCriticPolicy(
        observation_space=observation_space, 
        action_space=action_space, 
        net_arch=[256, 256],
        features_extractor_class = CustomCNN,
        features_extractor_kwargs = features_extractor_kwargs,
        share_features_extractor = True,
    normalize_images=False

)

# # model_path = "models/ppo_8x8_cnn/model"
# model_path = "models/cnn_selfplay/model_760w"
# # model_path = "models/model_460w"

# # model_path = "models/model"
# state_dict_path = model_path + '_state_dict.pt'

# policy_model.load_state_dict(torch.load(state_dict_path))

In [18]:
policy_model

MaskableActorCriticPolicy(
  (features_extractor): CustomCNN(
    (net): MyCnnNet(
      (conv_block): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
        (6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (8): ReLU()
        (9): Flatten(start_dim=1, end_dim=-1)
      )
    )
    (linear): Sequential(
      (0): Linear(in_features=8192, out_features=256, bias=True)
      (1): ReLU()
    )
  )
  (pi_features_extractor): CustomCNN(
    (net): MyCnnNet(
      (conv_block): Sequential(
        (0): Conv2d(3, 64, kernel_s

In [72]:
from reversi_model_train import ReversiModelTrain

obj = ReversiModelTrain(opponent_window_size=100,opponent_prob_decay_rate=0.9)
obj.opponent_distribution

self.archive_path: None


array([1.00000000e+00, 8.99997344e-01, 8.09994953e-01, 7.28992802e-01,
       6.56090865e-01, 5.90479123e-01, 5.31428554e-01, 4.78283042e-01,
       4.30452082e-01, 3.87404218e-01, 3.48661140e-01, 3.13792369e-01,
       2.82410476e-01, 2.54166772e-01, 2.28747439e-01, 2.05870039e-01,
       1.85280379e-01, 1.66749685e-01, 1.50072060e-01, 1.35062198e-01,
       1.21553322e-01, 1.09395333e-01, 9.84531439e-02, 8.86051733e-02,
       7.97419997e-02, 7.17651436e-02, 6.45859730e-02, 5.81247195e-02,
       5.23095913e-02, 4.70759760e-02, 4.23657222e-02, 3.81264937e-02,
       3.43111882e-02, 3.08774131e-02, 2.77870156e-02, 2.50056578e-02,
       2.25024358e-02, 2.02495360e-02, 1.82219262e-02, 1.63970774e-02,
       1.47547134e-02, 1.32765859e-02, 1.19462711e-02, 1.07489878e-02,
       9.67143279e-03, 8.70163330e-03, 7.82881376e-03, 7.04327617e-03,
       6.33629234e-03, 5.70000690e-03, 5.12735000e-03, 4.61195879e-03,
       4.14810670e-03, 3.73063982e-03, 3.35491963e-03, 3.01677145e-03,
      

In [57]:
obj._get_next_opponent_index()

3

In [69]:
0.96**200

0.00028460767526957515

In [64]:
1/0.0059

169.49152542372883

In [3]:
x=True
y=not x
y

False

In [129]:
# 0.8**20

0.011529215046068483

### model test

In [3]:
from model_test import game_play

# model_path = ".\models\cnn_selfplay\model_20w"
opponent_model_path="random"
opponent_model_path = ".\models\ppo_8x8_cnn_debug2\model_2600w"
# # opponent_model_path = ".\models\ppo_8x8_cnn_debug1\model_1000w"
# opponent_model_path = ".\models\ppo_8x8_resnet_lr_decay\model_1000w"
opponent_model_path = ".\models\cnn_selfplay\model_2275w"

# model_path = ".\models\ppo_8x8_cnn_debug2\model_2600w"
# model_path = ".\models\ppo_8x8_resnet_lr_decay\model_1000w"

model_path = ".\models\cnn_selfplay\model_2915w"

max_round=100

n_channels=3
board_size=8

verbose=0
deterministic=True

game_play(model_path, 
          opponent_model_path, 
          n_channels=n_channels, 
          max_round=max_round, 
          board_size=board_size, 
          verbose=verbose,
          deterministic=deterministic)


---- round:0 --------
total_win:1, total_failure: 0, total_equal:0

---- round:1 --------
total_win:2, total_failure: 0, total_equal:0

---- round:2 --------
total_win:3, total_failure: 0, total_equal:0

---- round:3 --------
total_win:4, total_failure: 0, total_equal:0

---- round:4 --------
total_win:5, total_failure: 0, total_equal:0

---- round:5 --------
total_win:6, total_failure: 0, total_equal:0

---- round:6 --------
total_win:7, total_failure: 0, total_equal:0

---- round:7 --------
total_win:8, total_failure: 0, total_equal:0

---- round:8 --------
total_win:9, total_failure: 0, total_equal:0

---- round:9 --------
total_win:10, total_failure: 0, total_equal:0

---- round:10 --------
total_win:11, total_failure: 0, total_equal:0

---- round:11 --------
total_win:12, total_failure: 0, total_equal:0

---- round:12 --------
total_win:13, total_failure: 0, total_equal:0

---- round:13 --------
total_win:14, total_failure: 0, total_equal:0

---- round:14 --------
total_win:15, to

KeyboardInterrupt: 

In [4]:
import torch
# from model_deploy.policies import CustomCNN, ActorCriticPolicy
from model_deploy.custom_feature_extractor import CustomCNN
from model_deploy.policies import MaskableActorCriticPolicy


backbone='cnn'
# backbone='res_net'

n_channels=3


features_extractor_kwargs=dict(features_dim=256,
                               backbone=backbone,
                               net_arch=[128, 128, 256],
#                                net_arch=[64, 128, 128],
                               kernel_size=3,
                               stride=1,
                               padding=1,
                               is_batch_norm=True
                               )

observation_space=(n_channels, 8, 8)
action_space = [observation_space[1]*observation_space[2]]

policy_model = MaskableActorCriticPolicy(
        observation_space=observation_space, 
        action_space=action_space, 
        net_arch=[256, 256],
        features_extractor_class = CustomCNN,
        features_extractor_kwargs = features_extractor_kwargs,
        share_features_extractor = True,
    normalize_images=False

)

policy_model2 = MaskableActorCriticPolicy(
        observation_space=observation_space, 
        action_space=action_space, 
        net_arch=[256, 256],
        features_extractor_class = CustomCNN,
        features_extractor_kwargs = features_extractor_kwargs,
        share_features_extractor = True,
    normalize_images=False

)
# model_path = "models/ppo_8x8_cnn/model"
# model_path = "models/cnn_selfplay/model_760w"
# model_path = "models/colab/resnet/model_200w"

# # model_path = "models/model"
# state_dict_path = model_path + '_state_dict.pt'

# policy_model.load_state_dict(torch.load(state_dict_path))

In [7]:
import time
import torch
from stable_baselines3 import PPO
from gym_reversi import ReversiEnv
from sb3_contrib.ppo_mask import MaskablePPO
# PolicyModel = PPO
PolicyModel = MaskablePPO


opponent_model="random"

# # opponent_model_path = ".\models\ppo_8x8_cnn_debug2\model_2600w"
# opponent_model_path = ".\models\cnn_selfplay\model_1200w"
# opponent_model = PolicyModel.load(opponent_model_path)

# state_dict_model_path = "models/colab/resnet/model_300w_state_dict.pt"
# policy_model2.load_state_dict(torch.load(state_dict_model_path))

# model_path = ".\models\ppo_8x8_resnet_lr_decay\model_1000w"
# model = PolicyModel.load(model_path)

# state_dict_model_path = "models/ppo_8x8_cnn_debug2/model_2600w_state_dict.pt"
# state_dict_model_path = "models/colab/resnet/model_215w_state_dict.pt"
state_dict_model_path = "models/cnn_selfplay/model_2915w_state_dict.pt"

policy_model.load_state_dict(torch.load(state_dict_model_path))

# policy_model.load_state_dict(torch.load("models/cnn_selfplay/model_760w_state_dict.pt"))
model = policy_model

# model = opponent_model
# opponent_model = policy_model


max_round=100

n_channels=3
board_size=8
verbose=0

deterministic = True
is_train=(not deterministic)

t0 = time.time()

n = 0
_win = 0
_failure = 0
_equal = 0

env = ReversiEnv(opponent=opponent_model, n_channels=n_channels, is_train=is_train,
                 board_size=board_size, verbose=verbose)

obs, info = env.reset()
while n < max_round:
    possible_actions = get_possible_actions(obs, player_color=env.player_color)
    action_masks = valid_action_mask(board_size=board_size, possible_actions=possible_actions)
    action, _states = model.predict(obs, deterministic=deterministic, action_masks=action_masks)
    action = int(action)
    obs, rewards, dones, truncated, info = env.step(action)
    #     print(f"---- round:{n} --------")
    #     print(f"action: {action}")
    #     env.render("human")
    if dones:
        print(f"---- round:{n} --------")
        #         env.render("human")
        obs, info = env.reset()
        n += 1
        if rewards > 0:
            _win += 1
        elif rewards < 0:
            _failure += 1
        else:
            _equal += 1
        print(f"total_win:{_win}, total_failure: {_failure}, total_equal:{_equal}\n")

_win_rate = round(_win/(_win + _failure) * 10000)/10000
print(f"win: {_win}, failure: {_failure}, equal_cnt: {_equal}, win_rate: {_win_rate}\n")
print(f" win_rate: {_win_rate}\n")
print(f"total time: {time.time() - t0}")


---- round:0 --------
total_win:1, total_failure: 0, total_equal:0

---- round:1 --------
total_win:2, total_failure: 0, total_equal:0

---- round:2 --------
total_win:3, total_failure: 0, total_equal:0

---- round:3 --------
total_win:4, total_failure: 0, total_equal:0

---- round:4 --------
total_win:5, total_failure: 0, total_equal:0

---- round:5 --------
total_win:6, total_failure: 0, total_equal:0

---- round:6 --------
total_win:7, total_failure: 0, total_equal:0

---- round:7 --------
total_win:8, total_failure: 0, total_equal:0

---- round:8 --------
total_win:9, total_failure: 0, total_equal:0

---- round:9 --------
total_win:10, total_failure: 0, total_equal:0

---- round:10 --------
total_win:11, total_failure: 0, total_equal:0

---- round:11 --------
total_win:12, total_failure: 0, total_equal:0

---- round:12 --------
total_win:13, total_failure: 0, total_equal:0

---- round:13 --------
total_win:14, total_failure: 0, total_equal:0

---- round:14 --------
total_win:15, to

In [ ]:
win: [483, 469], failure: [13, 24], equal_cnt: [4, 7], win_rate: [0.9738, 0.9513]
total win: 952, failure: 37, equal_cnt: 11, win_rate: 0.9626
train time: 477.33438515663147

### 模型加载

In [6]:
import numpy as np
from gym_reversi import ReversiEnv

def get_possible_actions(board, player_color):
    actions = []
    d = board.shape[-1]
    opponent_color = 1 - player_color
    for pos_x in range(d):
        for pos_y in range(d):
            if board[0, pos_x, pos_y] or board[1, pos_x, pos_y]:
                continue
            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    if dx == 0 and dy == 0:
                        continue
                    nx = pos_x + dx
                    ny = pos_y + dy
                    n = 0
                    if nx not in range(d) or ny not in range(d):
                        continue
                    while board[opponent_color, nx, ny] == 1:
                        tmp_nx = nx + dx
                        tmp_ny = ny + dy
                        if tmp_nx not in range(d) or tmp_ny not in range(d):
                            break
                        n += 1
                        nx += dx
                        ny += dy
                    if n > 0 and board[player_color, nx, ny] == 1:
                        action = pos_x * d + pos_y
                        if action not in actions:
                            actions.append(action)
    return actions

def set_possible_actions_place(board, possible_actions, channel_index=2):
    board[channel_index, :, :] = 0
    # possible_actions = ReversiEnv.get_possible_actions(board, player_color)
    possible_actions_coords = [ReversiEnv.action_to_coordinate(board, _action) for _action in possible_actions]
    for pos_x, pos_y in possible_actions_coords:
        board[channel_index, pos_x, pos_y] = 1
    return board

def get_test_observation(board_size=4, player_color=0):
    # init board setting
    N_CHANNELS = 4
    # channels： 0: 黑棋位置， 1: 白棋位置， 2: 当前可合法落子位置，3：player 颜色
    observation = np.zeros((N_CHANNELS, board_size, board_size), dtype=int)

    observation[3, :, :] = player_color

    centerL = int(board_size / 2 - 1)
    centerR = int(board_size / 2)
    # self.observation[2, :, :] = 1
    # self.observation[2, (centerL) : (centerR + 1), (centerL) : (centerR + 1)] = 0
    observation[0, centerR, centerL] = 1
    observation[0, centerL, centerR] = 1
    observation[1, centerL, centerL] = 1
    observation[1, centerR, centerR] = 1
    possible_actions = get_possible_actions(observation, player_color)

    # 设置主玩家合法位置
    set_possible_actions_place(observation, possible_actions)

    return observation


def action_to_coordinate(board, action):
    return action // board.shape[-1], action % board.shape[-1]


def valid_action_mask(board_size=8, possible_actions=[]):
    valid_actions = np.zeros((board_size**2, ), dtype=np.uint8)
    for idx in possible_actions:
        valid_actions[idx] = 1
    return valid_actions


In [27]:
import torch
from stable_baselines3 import PPO
from sb3_contrib.ppo_mask import MaskablePPO

# PolicyModel = PPO
PolicyModel = MaskablePPO

model_path = "models/ppo_8x8_backbone/model_80w"
# model_path = "models/model_460w"
state_dict_path = model_path + '_state_dict.pt'

sb3_model = PolicyModel.load(model_path)

torch.save(sb3_model.policy.state_dict(), state_dict_path)


In [13]:
sb3_model.policy

MaskableActorCriticPolicy(
  (features_extractor): CustomCNN(
    (net): MyResNet(
      (resnet_block): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (downsample): Sequential(
            (0): Conv2d(4, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_

In [5]:
sb3_model.policy.state_dict().keys()

odict_keys(['features_extractor.net.resnet_block.0.conv1.weight', 'features_extractor.net.resnet_block.0.bn1.weight', 'features_extractor.net.resnet_block.0.bn1.bias', 'features_extractor.net.resnet_block.0.bn1.running_mean', 'features_extractor.net.resnet_block.0.bn1.running_var', 'features_extractor.net.resnet_block.0.bn1.num_batches_tracked', 'features_extractor.net.resnet_block.0.conv2.weight', 'features_extractor.net.resnet_block.0.bn2.weight', 'features_extractor.net.resnet_block.0.bn2.bias', 'features_extractor.net.resnet_block.0.bn2.running_mean', 'features_extractor.net.resnet_block.0.bn2.running_var', 'features_extractor.net.resnet_block.0.bn2.num_batches_tracked', 'features_extractor.net.resnet_block.0.downsample.0.weight', 'features_extractor.net.resnet_block.0.downsample.1.weight', 'features_extractor.net.resnet_block.0.downsample.1.bias', 'features_extractor.net.resnet_block.0.downsample.1.running_mean', 'features_extractor.net.resnet_block.0.downsample.1.running_var', 'f

In [45]:
model.policy.action_net

Linear(in_features=256, out_features=16, bias=True)

In [1]:
import torch
# from model_deploy.policies import CustomCNN, ActorCriticPolicy
from model_deploy.custom_feature_extractor import CustomCNN
from model_deploy.policies import MaskableActorCriticPolicy

def build_model(state_dict_path, features_extractor_kwargs, observation_space=(4,8,8)):

    # features_extractor_kwargs=dict(features_dim=1024,
    #                                net_arch=[64, 128, 256],
    #                                # net_arch=[64, 128, 128],
    #                                # net_arch=[32, 64, 128],
    #                                kernel_size=3,
    #                                stride=1,
    #                                padding='same',
    #                                is_batch_norm=False),

    action_space = [observation_space[1]*observation_space[2]]

    policy_model = MaskableActorCriticPolicy(
                    observation_space = observation_space,
                    action_space = action_space,
                    net_arch=[256, 256],
                    features_extractor_class = CustomCNN,
                    features_extractor_kwargs = features_extractor_kwargs,
                    normalize_images= False,
    )

    return policy_model


In [15]:
import torch

# ppo_model = PPO.load("models/Reversi_ppo_4x4/model_100w")

# ## 保存模型
# torch.save(ppo_model.policy, 'models/Reversi_ppo_4x4/model_100w.pth')

# ## 读取模型
# pth_model = torch.load('models/Reversi_ppo_4x4/model_100w.pth')

observation = get_test_observation(board_size=8, player_color=0)
# observation

possible_actions = get_possible_actions(observation, player_color=0)

action_masks = valid_action_mask(board_size=8, possible_actions=possible_actions)


In [9]:

possible_actions

[19, 26, 37, 44]

In [17]:
action_masks

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=uint8)

In [18]:
action, _states = sb3_model.predict(observation, deterministic=True, action_masks=action_masks)
action
# action_to_coordinate(observation, action)

array(26, dtype=int64)

In [9]:
action

array(26, dtype=int64)

In [4]:
policy_model

MaskableActorCriticPolicy(
  (features_extractor): CustomCNN(
    (net): MyCnnNet(
      (conv_block): Sequential(
        (0): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
        (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (5): ReLU()
        (6): Flatten(start_dim=1, end_dim=-1)
      )
    )
    (linear): Sequential(
      (0): Linear(in_features=8192, out_features=256, bias=True)
      (1): ReLU()
    )
  )
  (pi_features_extractor): CustomCNN(
    (net): MyCnnNet(
      (conv_block): Sequential(
        (0): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU()
        (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU()
        (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (5): ReLU()
        (6): Flat

In [1]:
import torch
# from model_deploy.policies import CustomCNN, ActorCriticPolicy
from model_deploy.custom_feature_extractor import CustomCNN
from model_deploy.policies import MaskableActorCriticPolicy


# backbone='cnn'
backbone='res_net'

features_extractor_kwargs=dict(features_dim=256,
                               backbone=backbone,
#                                net_arch=[64, 128, 256],
                               net_arch=[64, 128, 128],
#                                net_arch=[32, 64, 128],
#                                net_arch=[32, 64, 64],
                               kernel_size=3,
                               stride=1,
                               padding=1,
                               is_batch_norm=False
                               )

observation_space=(4, 8, 8)
action_space = [observation_space[1]*observation_space[2]]

policy_model = MaskableActorCriticPolicy(
        observation_space=observation_space, 
        action_space=action_space, 
        net_arch=[256, 256],
        features_extractor_class = CustomCNN,
        features_extractor_kwargs = features_extractor_kwargs,
        share_features_extractor = True,
    normalize_images=False

)

# model_path = "models/ppo_8x8_cnn/model"
model_path = "models/ppo_8x8_backbone/model_440w"
# model_path = "models/model_460w"

# model_path = "models/model"
state_dict_path = model_path + '_state_dict.pt'

policy_model.load_state_dict(torch.load(state_dict_path))

<All keys matched successfully>

In [20]:

action, _states = policy_model.predict(observation, deterministic=True, action_masks=action_masks)
action

array([26], dtype=int64)

In [35]:
action.reshape(1)

array([37], dtype=int64)

In [38]:
int(action)

37

In [30]:
policy_model.state_dict()

OrderedDict([('features_extractor.cnn.0.weight',
              tensor([[[[ 5.8862e-02,  3.7363e-01, -1.3097e-01],
                        [-1.3117e-01, -1.4281e-01,  1.0642e-02],
                        [-3.2974e-01,  5.1945e-02, -1.7016e-01]],
              
                       [[-1.6550e-01,  2.2424e-01,  6.2481e-01],
                        [ 1.1638e-01,  6.0274e-02,  2.3141e-01],
                        [-1.1957e-01, -3.1702e-01,  2.2510e-01]],
              
                       [[-1.1080e-01,  3.4992e-01, -2.3739e-01],
                        [-8.8834e-02,  2.8504e-01,  1.4751e-01],
                        [ 1.0280e-01, -2.0961e-01, -1.3982e-01]],
              
                       [[-3.6928e-01,  2.0983e-01, -2.5590e-01],
                        [ 1.8024e-02,  4.0449e-01, -2.9480e-01],
                        [-1.0230e-01, -4.0876e-02,  4.6758e-01]]],
              
              
                      [[[ 4.3953e-02, -1.4663e-01,  3.7173e-01],
                        [ 

In [12]:
sb3_model.policy.state_dict()

OrderedDict([('features_extractor.cnn.0.weight',
              tensor([[[[ 5.8862e-02,  3.7363e-01, -1.3097e-01],
                        [-1.3117e-01, -1.4281e-01,  1.0642e-02],
                        [-3.2974e-01,  5.1945e-02, -1.7016e-01]],
              
                       [[-1.6550e-01,  2.2424e-01,  6.2481e-01],
                        [ 1.1638e-01,  6.0274e-02,  2.3141e-01],
                        [-1.1957e-01, -3.1702e-01,  2.2510e-01]],
              
                       [[-1.1080e-01,  3.4992e-01, -2.3739e-01],
                        [-8.8834e-02,  2.8504e-01,  1.4751e-01],
                        [ 1.0280e-01, -2.0961e-01, -1.3982e-01]],
              
                       [[-3.6928e-01,  2.0983e-01, -2.5590e-01],
                        [ 1.8024e-02,  4.0449e-01, -2.9480e-01],
                        [-1.0230e-01, -4.0876e-02,  4.6758e-01]]],
              
              
                      [[[ 4.3953e-02, -1.4663e-01,  3.7173e-01],
                        [ 